# Multi-modal news headline generator

### IMP: Before proceeding with the assignment, I wanted to point out that the task asked for Headline generation. But the datasets attached didn't have a headline column. so, i figured out you mean by headline > Summarization. and I went with this conclusion because of the datasets structure. it includes a normal text and then its summary.

#### also to add up to this at once, the datasets also did not include the Images. so I didn't use any images as you can see.

In [1]:
import kagglehub
import os
import pandas as pd
import numpy as np
from pathlib import Path

In [5]:
path = kagglehub.dataset_download("pariza/bbc-news-summary")
print("Path to dataset files:", path)

base_path = Path(path) / "BBC News Summary"
articles_path = base_path / "News Articles"
summaries_path = base_path / "Summaries"

Path to dataset files: C:\Users\CompuFast\.cache\kagglehub\datasets\pariza\bbc-news-summary\versions\2


### reading and extracting the data from the two subfolders I need, Into one pandas dataframe.

In [6]:
def read_file(filepath):
    for enc in ("utf-8", "cp1252", "latin-1"):
        try:
            with open(filepath, "r", encoding=enc) as f:
                return f.read().strip()
        except UnicodeDecodeError:
            continue
    return ""

records = []

for category in os.listdir(articles_path):
    category_articles_path = articles_path / category
    category_summaries_path = summaries_path / category
    
    if not category_articles_path.is_dir():
        continue
    
    for filename in os.listdir(category_articles_path):
        article_file = category_articles_path / filename
        summary_file = category_summaries_path / filename
        
        if article_file.is_file() and summary_file.is_file():
            article_text = read_file(article_file)
            summary_text = read_file(summary_file)
            
            records.append({
                "article_text": article_text,
                "summary_text": summary_text
            })

In [ ]:
df = pd.DataFrame(records)

print("DataFrame shape:", df.shape)
df.head()

DataFrame shape: (2225, 2)


,article_text,summary_text
0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...
1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...
2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...
3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ..."
4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...


### Comparing the actual text with its summary

In [8]:
print(df['article_text'][0] + "\n\n -------------" "\n"  + df['summary_text'][0])

Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL

## Preprocessing

In [9]:
df.isnull().sum()

article_text    0
summary_text    0
dtype: int64

In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CompuFast\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CompuFast\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CompuFast\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\CompuFast\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

#### note: in contrary of what we did in frequency based techniques, here we will actually need to retain the stop words and the words as they are.

In [11]:
def clean_text(doc):
    return re.sub(r'[^a-zA-Z\s]', '', doc)

def case_folding(tokens):
    return [word.lower() for word in tokens]

def tokenize(doc):
    return word_tokenize(doc)

In [12]:
def Preprocess(document):
    cleaned = clean_text(document)
    tokenized = tokenize(cleaned)
    tokenized = case_folding(tokenized)
    return ' '.join(tokenized)

In [13]:
df['article_text'] = df['article_text'].apply(Preprocess)
df['summary_text'] = df['summary_text'].apply(Preprocess)

In [14]:
text = df['article_text']
summary = df['summary_text']

In [15]:
print(text[0] + "\n\n -------------" "\n"  + summary[0])

ad sales boost time warner profit quarterly profits at us media giant timewarner jumped to bn m for the three months to december from m yearearlier the firm which is now one of the biggest investors in google benefited from sales of highspeed internet connections and higher advert sales timewarner said fourth quarter sales rose to bn from bn its profits were buoyed by oneoff gains which offset a profit dip at warner bros and less users for aol time warner said on friday that it now owns of searchengine google but its own internet business aol had has mixed fortunes it lost subscribers in the fourth quarter profits were lower than in the preceding three quarters however the company said aols underlying profit before exceptional items rose on the back of stronger internet advertising revenues it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aols existing customers for highspeed broadband timewarner also has to r

In [16]:
df.head()

,article_text,summary_text
0,ad sales boost time warner profit quarterly pr...,timewarner said fourth quarter sales rose to b...
1,dollar gains on greenspan speech the dollar ha...,the dollar has hit its highest level against t...
2,yukos unit buyer faces loan claim the owners o...,yukos owner menatep group says it will ask ros...
3,high fuel prices hit bas profits british airwa...,rod eddington bas chief executive said the res...
4,pernod takeover talk lifts domecq shares in uk...,pernod has reduced the debt it took on to fund...


 ## Transformer

### Here I used the pipeline library as Im familiar with and it is fast and efficient.
### Used this model because of its small size and still efficient performance tailored to this task

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization",model="google/flan-t5-small")
headline = summarizer(text[1], max_length=35, min_length=20, do_sample=False)
print(f"Original text: \n {text[1]} \n \n ------------- \n Headline:\n {headline[0]['summary_text']}")

Device set to use cuda:0


Original text: 
 dollar gains on greenspan speech the dollar has hit its highest level against the euro in almost three months after the federal reserve head said the us trade deficit is set to stabilise and alan greenspan highlighted the us governments willingness to curb spending and rising household savings as factors which may help to reduce it in late trading in new york the dollar reached against the euro from on thursday market concerns about the deficit has hit the greenback in recent months on friday federal reserve chairman mr greenspans speech in london ahead of the meeting of g finance ministers sent the dollar higher after it had earlier tumbled on the back of worsethanexpected us jobs data i think the chairmans taking a much more sanguine view on the current account deficit than hes taken for some time said robert sinche head of currency strategy at bank of america in new york hes taking a longerterm view laying out a set of conditions under which the current account defi

 ## LSTM

## For our LSTM modeling, we will be using an architecture of encoder and decoder in order to implement a summarization model.

In [18]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [35]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(text, summary,
                                       test_size=0.1, random_state=0, shuffle=True) 

## Pre-modeling procedures including
- applying the tokenizer
- padding the sequences
- defining the sizes

This code prepares the dataset for the seq2seq model:

Adds special **startseq** and *endseq* tokens to every summary so the model knows where a summary begins and ends.

Sets vocabulary limits (article_vocab_cap and summary_vocab_cap) to avoid learning rare words.

Defines **maximum sequence lengths** for articles and summaries.

Tokenizes the articles and summaries, replacing out-of-vocabulary words with <UNK>.

Calculates the actual vocabulary sizes to be used in the model.

In [ ]:
START, END = "startseq", "endseq"

y_train_proc = [f"{START} {s} {END}" for s in y_train]
y_test_proc  = [f"{START} {s} {END}" for s in y_test]

article_vocab_cap = 5000     
summary_vocab_cap = 1500
article_maxlen = 80          
summary_maxlen = 20          
latent_dim = 64              
embedding_dim = 50

article_tok = Tokenizer(num_words=article_vocab_cap, oov_token="<UNK>")
article_tok.fit_on_texts(x_train)

summary_tok = Tokenizer(num_words=summary_vocab_cap, oov_token="<UNK>")
summary_tok.fit_on_texts(y_train_proc)

article_vocab_size = min(article_vocab_cap, len(article_tok.word_index) + 1)
summary_vocab_size = min(summary_vocab_cap, len(summary_tok.word_index) + 1)
print("Article vocab:", article_vocab_size, "Summary vocab:", summary_vocab_size)

Article vocab: 5000 Summary vocab: 1500


This code transforms the tokenized text into model-ready input:

* **Converts** articles and summaries into integer sequences using the fitted tokenizers.
* **Pads or truncates** sequences to fixed lengths (article_maxlen and summary_maxlen).
* **Splits** summaries into:

  * decoder_input > all tokens except the last (fed into the decoder)
  * decoder_target > all tokens except the first (what the decoder should predict)
* **Creates a mask** to ignore padding (0s) when computing the loss.


In [ ]:
# Train
X_train_seq = article_tok.texts_to_sequences(x_train)
Y_train_seq = summary_tok.texts_to_sequences(y_train_proc)
X_train_pad = pad_sequences(X_train_seq, maxlen=article_maxlen, padding='post', truncating='post')
Y_train_pad = pad_sequences(Y_train_seq, maxlen=summary_maxlen, padding='post', truncating='post')

# Test  
X_test_seq  = article_tok.texts_to_sequences(x_test)
Y_test_seq  = summary_tok.texts_to_sequences(y_test_proc)
X_test_pad  = pad_sequences(X_test_seq,  maxlen=article_maxlen, padding='post', truncating='post')
Y_test_pad  = pad_sequences(Y_test_seq,  maxlen=summary_maxlen, padding='post', truncating='post')

decoder_input  = Y_train_pad[:, :-1]     
decoder_target = Y_train_pad[:, 1:]      
print("Train shapes -> X:", X_train_pad.shape, "DecIn:", decoder_input.shape, "DecTgt:", decoder_target.shape)

decoder_target_mask = (decoder_target != 0).astype(np.float32)

Train shapes -> X: (2002, 80) DecIn: (2002, 19) DecTgt: (2002, 19)


## Architecture: 
It’s a sequence-to-sequence LSTM model for text summarization:
the encoder reads the article and produces hidden states,
the decoder uses those states to predict the summary word-by-word,
and a softmax layer outputs the probability for each word in the vocabulary.

In [38]:
encoder_inputs = Input(shape=(article_maxlen,))
enc_emb = Embedding(article_vocab_cap, embedding_dim, mask_zero=True)(encoder_inputs)
_, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)

decoder_inputs = Input(shape=(summary_maxlen-1,))
dec_emb = Embedding(summary_vocab_cap, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_outputs = LSTM(latent_dim, return_sequences=True)(dec_emb, initial_state=[state_h, state_c])
outputs = Dense(summary_vocab_cap, activation='softmax')(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 80, 50)    │    250,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_8         │ (None, 80)        │          0 │ input_layer_2[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 19, 50)    │     75,000 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 64),      │     29,440 │ embedding_2[0][0… │
│                     │ (None, 64),       │            │ not_equal_8[0][0] │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 19, 64)    │     29,440 │ embedding_3[0][0… │
│                     │                   │            │ lstm_2[0][1],     │
│                     │                   │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 19, 1500)  │     97,500 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 481,380 (1.84 MB)

 Trainable params: 481,380 (1.84 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1)
reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=3e-5, verbose=1)
ckpt       = ModelCheckpoint("best_seq2seq.keras", monitor='val_loss', save_best_only=True, verbose=1)

history = model.fit(
    [X_train_pad, decoder_input],
    decoder_target,                          
    sample_weight=decoder_target_mask,        
    batch_size=32,
    epochs=20,    
    validation_split=0.15,
    callbacks=[early_stop, reduce_lr, ckpt],
    verbose=1
)

Epoch 1/20
53/54 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1922 - loss: 6.9136
Epoch 1: val_loss improved from inf to 5.09529, saving model to best_seq2seq.keras
54/54 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.1933 - loss: 6.8892 - val_accuracy: 0.2280 - val_loss: 5.0953 - learning_rate: 0.0010
Epoch 2/20
53/54 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.2352 - loss: 5.0356
Epoch 2: val_loss improved from 5.09529 to 5.07670, saving model to best_seq2seq.keras
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.2351 - loss: 5.0359 - val_accuracy: 0.2280 - val_loss: 5.0767 - learning_rate: 0.0010
Epoch 3/20
53/54 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.2274 - loss: 5.0598
Epoch 3: val_loss did not improve from 5.07670
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.2276 - loss: 5.0588 - val_accuracy: 0.2280 - val_loss: 5.0774 - learning_rate: 0.0010
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2312 - loss: 5.0206
Epoch 4: val_loss impro

### Evaluation

In [55]:
from nltk.translate.bleu_score import corpus_bleu
import numpy as np

def predict_summary(model, input_seq, summary_tok, max_len=20):
    start_token = summary_tok.word_index.get('startseq', 1)
    end_token = summary_tok.word_index.get('endseq', 2)
    
    decoder_input = np.zeros((1, max_len-1))
    decoder_input[0, 0] = start_token
    
    for i in range(1, max_len-1):
        pred = model.predict([input_seq, decoder_input], verbose=0)
        next_token = np.argmax(pred[0, i-1, :])
        
        if next_token == end_token or next_token == 0:
            break
        decoder_input[0, i] = next_token
    
    reverse_word_map = {v: k for k, v in summary_tok.word_index.items()}
    words = []
    for token in decoder_input[0]:
        if token != 0 and token in reverse_word_map:
            word = reverse_word_map[token]
            if word not in ['startseq', 'endseq']:
                words.append(word)
    return words

def evaluate_bleu(model, X_test, y_test, summary_tok, num_samples=100):
    generated = []
    references = []
    
    if hasattr(y_test, 'iloc'):
        y_test = y_test.tolist()
    
    print(f"Generating {num_samples} summaries...")
    for i in range(min(num_samples, len(X_test))):
        if i % 20 == 0:
            print(f"Progress: {i}/{num_samples}")
        
        pred_words = predict_summary(model, X_test[i:i+1], summary_tok)
        ref_words = y_test[i].split()
        
        generated.append(pred_words)
        references.append([ref_words])
    
    bleu_score = corpus_bleu(references, generated)
    print(f"\nBLEU Score: {bleu_score:.4f}")
    
    for i in range(3):
        print(f"\nExample {i+1}:")
        print(f"Generated: {' '.join(generated[i])}")
        print(f"Reference: {' '.join(references[i][0])}")
    
    return bleu_score

import tensorflow as tf
model = tf.keras.models.load_model("best_seq2seq.keras")
bleu_score = evaluate_bleu(model, X_test_pad, y_test, summary_tok)

Generating 100 summaries...
Progress: 0/100
Progress: 20/100
Progress: 40/100
Progress: 60/100
Progress: 80/100

BLEU Score: 0.0000

Example 1:
Generated: the <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
Reference: after years of a sagging stock price and a sevenyear hiatus from the ftse one of britains venerable manufacturers has returned to the vaunted index forbes saidthe sugar group had been absent from the ftse for seven years until mr ferguson helped it return to growthtate lyles chief executive has been named european businessman of the year by a leading business magazineiain ferguson was awarded the title by us publication forbes for returning one of the uks venerable manufacturers to the countrys top companies

Example 2:
Generated: the <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
Reference: record numbers of warcraft fans are settling in the games online worldhalo

# Conclusion:
## The difference is clear here between the Transformer and the LSTM.
## As the dataset is extremely small for such task with only 2225 samples. There is no space for even splitting on three train, dev and test.
## The transformer was better in everything because it is pre-trained on huge amount of data and because of course transformers are generally better because of the way they use attention as we know ( Attention is all we need ). so, In general Transformers are way better.